## Agents on Kubeflow 🤓

In this tutorial we will be training a reinforcement learning agent from the [tensorflow/agents](https://github.com/tensorflow/agents) project on Kubernetes using [Kubeflow](https://github.com/google/kubeflow).

The task the agent will be learning to perform is to operate a Kuka Robotics arm simulated in the OpenAI Gym Bullet Physics 'KukaBulletEnv-v0' environment. Feel free to [skip to the end](http://localhost:8888/notebooks/kubeflow-rl/apps/agents_ppo/demo.ipynb#Rendering-the-model) to see what this will look like!

### Setup

We need to create a Google Cloud Storage bucket to store job logs as well as a unique subdirectory of that bucket to store logs for this particular run. With the following we first create the GCS bucket then generate the path of a log dir to use in a later step.

Set the variables below to a project and bucket suitable for your use.

In [1]:
# GCP project to use
PROJECT="kubeflow-rl"

# Bucket to use
BUCKET=PROJECT+"-kf"

# K8s cluster to use
CLUSTER="kubeflow-5fc52116"
ZONE="us-east1-d"
NAMESPACE="rl"

# Find the path to the workspace ml-app dir 
import subprocess, os
def find_workspace_root(cwd=None):
    if cwd is None:
        cwd = subprocess.check_output(["pwd"]).strip()
    files = subprocess.check_output(["ls", cwd])
    if "WORKSPACE" in files:
        return cwd
    else:
        cwd = '/'.join(os.path.split(cwd)[:-1])
        return find_workspace_root(cwd)
ML_APP_DIR=os.path.join(find_workspace_root(), "ml-app")

# Needed for launching tensorboard
SECRET_NAME = "gcp-credentials"

**Attention:** You will need GCP credentials to access the cluster and GCP resources.

If you're running this on your local machine you can authenticate for the GCP project you specified above in the usual way (i.e. `gcloud auth login` followed by `gcloud config set project <your-project-name>`; in this case skip the following step.

If you're running this on JupyterLab you can do the following to provide the right credentials:

- Create a service account and download the private key
    - TODO: Clarity about which roles are needed
- Use JupyterLab to upload the service account to your pod
- Set the path to your service account in the cell below and then execute it to activate the service account

In [165]:
KEY_FILE="<path to your GCS key file json>"
!gcloud auth activate-service-account --key-file={KEY_FILE}

Activated service account credentials for: [cwbeitel-kubeflow-rl@kubeflow-rl.iam.gserviceaccount.com]


In [166]:
!gsutil mb -p {PROJECT} gs://{BUCKET}

Creating gs://kubeflow-rl-kf/...
AccessDeniedException: 403 cwbeitel-kubeflow-rl@kubeflow-rl.iam.gserviceaccount.com does not have storage.buckets.create access to project 991277910492.


In [2]:
!gcloud container clusters --project={PROJECT} --zone={ZONE} get-credentials {CLUSTER}

Fetching cluster endpoint and auth data.
kubeconfig entry generated for kubeflow-5fc52116.


In [3]:
!kubectl create namespace {NAMESPACE}

Error from server (AlreadyExists): namespaces "rl" already exists


Download and install ksonnet if needed

In [11]:
!if ! [[ $(which ks) ]]; then mkdir -p ${HOME}/bin && curl -L -o ${HOME}/bin/ks "https://github.com/ksonnet/ksonnet/releases/download/v0.8.0/ks-linux-amd64" && chmod a+rx ${HOME}/bin/ks; fi

If running on GCP (or possibly another Cloud) you probably need to create a key with credentials to use for your job

In [57]:
SECRET_FILE_NAME="secret.json"
!kubectl create -n {NAMESPACE} secret generic {SECRET_NAME} --from-file={SECRET_FILE_NAME}={KEY_FILE}

secret "gcp-credentials" created


### Training

The objective of the training phase is to learn the parameterization of our model that confers a high level of performance on the provided task. Here we'll launch and monitor a job.

#### Launching the TFJob

We'll use [ksonnet](https://ksonnet.io/) to parameterize and apply a TFJob configuration (i.e. run a job). Here you can change the image to be a custom job image, such as one built and deployed with build.sh, or use the one provided here if you only want to change parameters. Below we'll display the templated job YAML for reference.

In [4]:
# Check your cluster and see if that matches one of the existing ksonnet environments
# You want the kubernetes master server to be the same as the server listed for the ks environment
!kubectl cluster-info
!ks env list

Kubernetes master is running at https://35.185.119.177
GLBCDefaultBackend is running at https://35.185.119.177/api/v1/namespaces/kube-system/services/default-http-backend/proxy
Heapster is running at https://35.185.119.177/api/v1/namespaces/kube-system/services/heapster/proxy
KubeDNS is running at https://35.185.119.177/api/v1/namespaces/kube-system/services/kube-dns/proxy
kubernetes-dashboard is running at https://35.185.119.177/api/v1/namespaces/kube-system/services/kubernetes-dashboard/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.
ERROR lstat /Users/cb/environments: no such file or directory


In [6]:
import datetime
import uuid
import os

os.chdir(ML_APP_DIR)

HPARAM_SET="pybullet-kuka"

now=datetime.datetime.now()

JOB_SALT=now.strftime("%m%d-%H%M") + "-" + uuid.uuid4().hex[0:4]
TRAIN_JOB_NAME=HPARAM_SET + "-" + JOB_SALT
LOG_DIR="gs://{0}/jobs/{1}".format(BUCKET, TRAIN_JOB_NAME)

IMAGE="gcr.io/kubeflow-rl/agents:agents-0202-1020-01ec"

!ks param set agents env "KukaBulletEnv-v0"

!ks param set agents run_mode train
!ks param set agents gcp_project kubeflow-rl
!ks param set agents num_cpu 31
!ks param set agents num_agents 30
!ks param set agents sync_replicas False
!ks param set agents steps 4e7
!ks param set agents update_every 30
!ks param set agents max_length 1000
!ks param set agents eval_episodes 25

# Trigger an async render job every 10 minutes
!ks param set agents render_secs 600

!ks param set agents algorithm "agents.algorithms.ppo.ppo.PPO"
!ks param set agents network "agents.scripts.networks.feed_forward_gaussian"

!ks param set agents job_tag {JOB_SALT}
!ks param set agents logdir {LOG_DIR}
!ks param set agents name {TRAIN_JOB_NAME}
!ks param set agents image {IMAGE}
!ks show default -c agents

!ks apply gke -c agents

INFO  Parameter 'env' successfully set to '"KukaBulletEnv-v0"' for component 'agents'
INFO  Parameter 'run_mode' successfully set to '"train"' for component 'agents'
INFO  Parameter 'gcp_project' successfully set to '"kubeflow-rl"' for component 'agents'
INFO  Parameter 'num_cpu' successfully set to '31' for component 'agents'
INFO  Parameter 'num_agents' successfully set to '30' for component 'agents'
INFO  Parameter 'sync_replicas' successfully set to '"False"' for component 'agents'
INFO  Parameter 'steps' successfully set to '4e7' for component 'agents'
INFO  Parameter 'update_every' successfully set to '30' for component 'agents'
INFO  Parameter 'max_length' successfully set to '1000' for component 'agents'
INFO  Parameter 'eval_episodes' successfully set to '25' for component 'agents'
INFO  Parameter 'render_secs' successfully set to '600' for component 'agents'
INFO  Parameter 'algorithm' successfully set to '"agents.algorithms.ppo.ppo.PPO"' for component 'agents'
INFO  Paramete

Now we can list tfjobs and see that a job has been created.

In [7]:
!kubectl get tfjobs -n {NAMESPACE} -o yaml {TRAIN_JOB_NAME}

apiVersion: tensorflow.org/v1alpha1
kind: TfJob
metadata:
  clusterName: ""
  creationTimestamp: 2018-02-02T18:53:36Z
  generation: 0
  name: pybullet-kuka-0202-1053-3a98
  namespace: rl
  resourceVersion: "4043397"
  selfLink: /apis/tensorflow.org/v1alpha1/namespaces/rl/tfjobs/pybullet-kuka-0202-1053-3a98
  uid: 5ff5fc69-084a-11e8-b604-42010af00218
spec:
  RuntimeId: zu80
  replicaSpecs:
  - IsDefaultPS: false
    replicas: 1
    template:
      metadata:
        creationTimestamp: null
      spec:
        containers:
        - args:
          - --run_mode=train
          - --logdir=gs://kubeflow-rl-kf/jobs/pybullet-kuka-0202-1053-3a98
          - --hparam_set_id=pybullet_kuka_ff
          - --run_base_tag=0e90193e
          - --sync_replicas=False
          - --num_gpus=0
          - --algorithm=agents.algorithms.ppo.ppo.PPO
          - --num_agents=30
          - --eval_episodes=25
          - --env=KukaBulletEnv-v0
          - --max_length=1000
          - --steps=40000000
        

#### Monitoring training

The IDs, status, and other metadata of pods involved in the training job can be displayed using the following:

In [8]:
!kubectl get pods -n rl --show-all

NAME                                               READY     STATUS        RESTARTS   AGE
pybullet-kuka-0202-1044-fe76-master-v79e-0-79mc2   1/1       Terminating   0          9m
pybullet-kuka-0202-1052-f773-master-qcpn-0-f8brv   1/1       Terminating   0          1m
pybullet-kuka-0202-1053-3a98-master-zu80-0-cdn9s   1/1       Running       0          2s
ubuntu                                             1/1       Running       0          9d


Logs from a specific pod can be displayed with the following (or streamed by adding the --follow flag):

In [9]:
!kubectl -n {NAMESPACE} get pods -o yaml pybullet-kuka-0202-1053-3a98-master-zu80-0-cdn9s

apiVersion: v1
kind: Pod
metadata:
  annotations:
    kubernetes.io/created-by: |
      {"kind":"SerializedReference","apiVersion":"v1","reference":{"kind":"Job","namespace":"rl","name":"pybullet-kuka-0202-1053-3a98-master-zu80-0","uid":"5ffa2649-084a-11e8-b604-42010af00218","apiVersion":"batch","resourceVersion":"4043392"}}
  creationTimestamp: 2018-02-02T18:53:36Z
  generateName: pybullet-kuka-0202-1053-3a98-master-zu80-0-
  labels:
    controller-uid: 5ffa2649-084a-11e8-b604-42010af00218
    job-name: pybullet-kuka-0202-1053-3a98-master-zu80-0
    job_type: MASTER
    runtime_id: zu80
    task_index: "0"
    tensorflow.org: ""
    tf_job_name: pybullet-kuka-0202-1053-3a98
  name: pybullet-kuka-0202-1053-3a98-master-zu80-0-cdn9s
  namespace: rl
  ownerReferences:
  - apiVersion: batch/v1
    blockOwnerDeletion: true
    controller: true
    kind: Job
    name: pybullet-kuka-0202-1053-3a98-master-zu80-0
    uid: 5ffa2649-084a-11e8-b604-42010af00218
  resourceVersion: "4043403"
  selfL

In [10]:
TRAIN_JOB_NAME

'pybullet-kuka-0202-1053-3a98'

Obtain the ID of the master pod and print logs

In [12]:
import subprocess
master_pod = subprocess.check_output(["kubectl", "-n", NAMESPACE, "get", "pods", "--selector=tf_job_name=" + TRAIN_JOB_NAME,
                                      "-o", "jsonpath='{.items[*].metadata.name}'"]).decode("utf-8")
print(master_pod)
!kubectl logs -n {NAMESPACE} {master_pod}

'pybullet-kuka-0202-1053-3a98-master-zu80-0-cdn9s'
/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:Start a new run and write summaries and checkpoints to gs://kubeflow-rl-kf/jobs/pybullet-kuka-0202-1053-3a98.
{'algorithm': <class 'agents.algorithms.ppo.ppo.PPO'>,
 'debug': True,
 'discount': 0.995,
 'dump_dependency_versions': False,
 'env': 'KukaBulletEnv-v0',
 'env_processes': True,
 'eval_episodes': 25,
 'hparam_set_id': 'pybullet_kuka_ff',
 'init_logstd': -1,
 'init_mean_factor': 0.1,
 'init_output_factor': 0.1,
 'init_std': 0.35,
 'kl_cutoff_coef': 1000,
 'kl_cutoff_factor': 2,
 'kl_init_penalty': 1,
 'kl_target': 0.01,
 'learning_rate': 0.0001,
 'log_device_placement': False,
 'logdir': 'gs://kubeflow-rl-kf/jobs/

#### Launching tensorboard

In [59]:
!ks param set tensorboard name {TRAIN_JOB_NAME}
!ks param set tensorboard namespace {NAMESPACE}
!ks param set tensorboard log_dir {LOG_DIR}
!ks param set tensorboard secret {SECRET_NAME}
!ks param set tensorboard secret_file_name {SECRET_FILE_NAME}
!ks show default -c tensorboard

!ks apply gke -c tensorboard

INFO  Parameter 'name' successfully set to '"pybullet-kuka-0123-1053-4837"' for component 'tensorboard'
INFO  Parameter 'namespace' successfully set to '"rl"' for component 'tensorboard'
INFO  Parameter 'log_dir' successfully set to '"gs://kubeflow-rl-kf/jobs/pybullet-kuka-0123-1053-4837"' for component 'tensorboard'
INFO  Parameter 'secret' successfully set to '"gcp-credentials"' for component 'tensorboard'
INFO  Parameter 'secret_file_name' successfully set to '"secret.json"' for component 'tensorboard'
---
apiVersion: apps/v1beta1
kind: Deployment
metadata:
  name: pybullet-kuka-0123-1053-4837-tb
  namespace: rl
spec:
  replicas: 1
  template:
    metadata:
      labels:
        app: tensorboard
        tb-job: pybullet-kuka-0123-1053-4837
      name: pybullet-kuka-0123-1053-4837
      namespace: rl
    spec:
      containers:
      - command:
        - /usr/local/bin/tensorboard
        - --logdir=gs://kubeflow-rl-kf/jobs/pybullet-kuka-0123-1053-4837
        - --port=80
        env

### Connecting to Tensorboard

To connect to tensorboard use kubectl proxy and then access it and the url given by the URL returned by evaluating the next cell

In [60]:
PROXY_PORT=8001
url=("http://127.0.0.1:{proxy_port}/api/v1/proxy/namespaces/{namespace}/services/{service_name}:80/".format(
    proxy_port=PROXY_PORT, namespace=NAMESPACE, service_name=TRAIN_JOB_NAME + "-tb"))
print(url)

http://127.0.0.1:8001/api/v1/proxy/namespaces/rl/services/pybullet-kuka-0123-1053-4837-tb:80/


### Deleting jobs

In [34]:
!kubectl delete tfjobs -n {NAMESPACE} {TRAIN_JOB_NAME}

tfjob "pybullet-kuka-0123-1000-3ad9" deleted


### Rendering the model

#### Initiating a rendering job directly

Launching a rendering job is as simple as the following:

In [13]:
import datetime
import uuid
import os

os.chdir(ML_APP_DIR)

now=datetime.datetime.now()

JOB_SALT=now.strftime("%m%d-%H%M") + "-" + uuid.uuid4().hex[0:4]
RENDER_JOB_NAME="render-" + JOB_SALT
IMAGE="gcr.io/kubeflow-rl/agents:agents-0202-1020-01ec"

!ks param set agents name {RENDER_JOB_NAME}

# Note for rendering our resource needs are much smaller
!ks param set agents num_cpu 1

# By default the logdir for the last submitted job will be rendered. To render for
# an arbitrary logdir, specify it here.
# ks param set agents logdir gs://kubeflow-rl-kf/jobs/kuka-8113eb39

!ks param set agents run_mode render
!ks apply gke -c agents

INFO  Parameter 'name' successfully set to '"render-0202-1054-9dfa"' for component 'agents'
INFO  Parameter 'num_cpu' successfully set to '1' for component 'agents'
INFO  Parameter 'run_mode' successfully set to '"render"' for component 'agents'
INFO  Updating tfjobs rl.render-0202-1054-9dfa
INFO  Creating non-existent tfjobs rl.render-0202-1054-9dfa


In [14]:
%%bash
kubectl get pods -n rl --show-all

NAME                                               READY     STATUS    RESTARTS   AGE
pybullet-kuka-0202-1053-3a98-master-zu80-0-cdn9s   1/1       Running   0          48s
render-0202-1054-9dfa-master-1sdp-0-nkvq5          1/1       Running   0          4s
ubuntu                                             1/1       Running   0          9d


In [21]:
import subprocess
master_pod = subprocess.check_output(["kubectl", "-n", NAMESPACE, "get", "pods", "--selector=tf_job_name=" + RENDER_JOB_NAME,
                                      "-o", "jsonpath='{.items[*].metadata.name}'"]).decode("utf-8")
print(master_pod)
!kubectl logs -n {NAMESPACE} {master_pod}

'render-0202-1054-9dfa-master-1sdp-0-nkvq5'
/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:Resume run and write summaries and checkpoints to gs://kubeflow-rl-kf/jobs/pybullet-kuka-0202-1053-3a98.
INFO:tensorflow:Resume run and write summaries and checkpoints to gs://kubeflow-rl-kf/jobs/pybullet-kuka-0202-1053-3a98.
{'algorithm': <class 'agents.algorithms.ppo.ppo.PPO'>,
 'debug': True,
 'discount': 0.995,
 'dump_dependency_versions': False,
 'env': 'KukaBulletEnv-v0',
 'env_processes': True,
 'eval_episodes': 25,
 'hparam_set_id': 'pybullet_kuka_ff',
 'init_logstd': -1,
 'init_mean_factor': 0.1,
 'init_output_factor': 0.1,
 'init_std': 0.35,
 'kl_cutoff_coef': 1000,
 'kl_cutoff_factor': 2,
 'kl_init_penalty': 1,
 'kl_t

List the contents of the render subdir that was created by the render job (when finished):

In [26]:
!gsutil ls {LOG_DIR}/render

gs://kubeflow-rl-kf/jobs/pybullet-kuka-0202-1053-3a98/render/openaigym.episode_batch.0.45.stats.json
gs://kubeflow-rl-kf/jobs/pybullet-kuka-0202-1053-3a98/render/openaigym.manifest.0.45.manifest.json
gs://kubeflow-rl-kf/jobs/pybullet-kuka-0202-1053-3a98/render/openaigym.video.0.45.video000000.meta.json
gs://kubeflow-rl-kf/jobs/pybullet-kuka-0202-1053-3a98/render/openaigym.video.0.45.video000000.mp4
gs://kubeflow-rl-kf/jobs/pybullet-kuka-0202-1053-3a98/render/openaigym.video.0.45.video000001.meta.json
gs://kubeflow-rl-kf/jobs/pybullet-kuka-0202-1053-3a98/render/openaigym.video.0.45.video000001.mp4
gs://kubeflow-rl-kf/jobs/pybullet-kuka-0202-1053-3a98/render/openaigym.video.0.45.video000002.meta.json
gs://kubeflow-rl-kf/jobs/pybullet-kuka-0202-1053-3a98/render/openaigym.video.0.45.video000002.mp4
gs://kubeflow-rl-kf/jobs/pybullet-kuka-0202-1053-3a98/render/openaigym.video.0.45.video000003.meta.json
gs://kubeflow-rl-kf/jobs/pybullet-kuka-0202-1053-3a98/render/openaigym.video.0.45.video000

In [27]:
!mkdir /tmp/agents-render
!gsutil cp `gsutil ls {LOG_DIR}/render | grep mp4 | head -n1` /tmp/agents-render/render.mp4

Copying gs://kubeflow-rl-kf/jobs/pybullet-kuka-0202-1053-3a98/render/openaigym.video.0.45.video000000.mp4...
- [1 files][196.6 KiB/196.6 KiB]                                                
Operation completed over 1 objects/196.6 KiB.                                    


#### Inspecting the result

When the job is complete there will be a subdirectory of the log dir named "render" with a number of short videos of episodes of the agent performing the grasping task. Here's an example of what one of those looks like in a well-trained model.

In [28]:
import io
import base64
from IPython.display import HTML

mp4_path = '/tmp/agents-render/render.mp4'

video = io.open(mp4_path, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))

### Great job! 🎉🎉🎉

If this is your first time working with these technologies you might be interested in some suggestions of good next steps. Here are some ideas:
- Try training with some other learning environments (from the ID fields [here](https://github.com/bulletphysics/bullet3/blob/master/examples/pybullet/gym/pybullet_envs/__init__.py)) and tweet your results! E.g.
    - RacecarBulletEnv-v0
    - MinitaurBulletDuckEnv-v0
    - HalfCheetahBulletEnv-v0
- Take a shot at implementing your own gym learning environment and repeat the above.